In [23]:
import numpy as np
import gym # contains out Taxi v3 environment
import random

# good walkthrough: https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/
# original google colab: https://colab.research.google.com/gist/simoninithomas/466c81aa1c2a07dd14793240c6d033c5/q-learning-with-taxi-v3.ipynb#scrollTo=20tSdDbxxK_H

Using OpenAI gym

It has this exact environment already built for us.

Gym provides different game environments which we can plug into our code and test an agent. The library takes care of API for providing all the information that our agent would require, like possible actions, score, and current state. We just need to focus just on the algorithm part for our agent.

We'll be using the Gym environment called Taxi-V3, which all of the details explained above were pulled from. The objectives, rewards, and actions are all the same.

In [86]:
# create environment and render it so we can see it
env = gym.make("Taxi-v3")
env.reset()
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




Our environment looks like this:

It's a 5x5 grid world
Our taxi is spawned randomly in a square.
The passenger is spawned randomly in one of the 4 possible locations (R, B, G, Y) and wishes to go in one of the 4 possibles locations too.

In [26]:
# investigate observation space and actions space

state_space = env.observation_space.n
print("There are ", state_space, " possible states")
action_space = env.action_space.n
print("There are ", action_space, " possible actions")

There are  500  possible states
There are  6  possible actions


In [24]:
# STATE SPACE

# The State Space is the set of all possible situations our taxi could inhabit. The state should contain useful information the agent needs to make the right action.
# 500 possible states that the board can be in as this includes all combinations of passenger locations, destination locations and 5 x 5 grid that the car could be in = 5 * 5 * 5 * 4 = 500

# ACTION SPACE

# The agent encounters one of the 500 states and it takes an action. 
# The action in our case can be to move in a direction or decide to pickup/dropoff a passenger.
# 6 poss actions: SOUTH NORTH EAST WEST PICKUP DROP-OFF

# the taxi cannot perform certain actions in certain states due to walls. In the environment's code, we will simply provide a -1 penalty for every wall hit and the taxi won't move anywhere. This will just rack up penalties causing the taxi to consider going around the wall.

### AS RL works by rewards lets have a look at them:
### Rewards
Since the agent (the imaginary driver) is reward-motivated and is going to learn how to control the cab by trial experiences in the environment, we need to decide the rewards and/or penalties and their magnitude accordingly. Here a few points to consider:

 - The agent should receive a high positive reward for a successful dropoff because this behavior is highly desired +20
 - The agent should be penalized if it tries to drop off a passenger in wrong locations -10
 - The agent should get a slight negative reward for not making it to the destination after every time-step. "Slight" negative because we would prefer our agent to reach late instead of making wrong moves trying to reach to the destination as fast as possible -1

The core gym interface is `env`, which is the unified environment interface. The following are the env methods that would be quite helpful to us:

 - `env.reset`: Resets the environment and returns a random initial state.
 - `env.step(action)`: Step the environment by one timestep. Returns
    - **observation**: Observations of the environment
    - **reward**: If your action was beneficial or not
    - **done**: Indicates if we have successfully picked up and dropped off a passenger, also called one episode
    - **info**: Additional info such as performance and latency for debugging purposes
 - `env.render`: Renders one frame of the environment (helpful in visualizing the environment)

Note: We are using the `.env on` the end of `make` to avoid training stopping at 200 iterations, which is the default for the new version of Gym (reference).



**Reminder of our problem**

 - Here's our restructured problem statement (from Gym docs):

*"There are 4 locations (labeled by different letters), and our job is to pick up the passenger at one location and drop him off at another. We receive +20 points for a successful drop-off and lose 1 point for every time-step it takes. There is also a 10 point penalty for illegal pick-up and drop-off actions."*

Let's dive more into the environment.

In [46]:
print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

Action Space Discrete(6)
State Space Discrete(500)


 - The filled square represents the taxi, which is yellow without a passenger and green with a passenger.
 - The pipe ("|") represents a wall which the taxi cannot cross.
 - R, G, Y, B are the possible pickup and destination locations. The blue letter represents the current passenger pick-up location, and the purple letter is the current destination.
 
 As verified by the prints, we have an Action Space of size 6 and a State Space of size 500. As you'll see, our RL algorithm won't need any more information than these two things. All we need is a way to identify a state uniquely by assigning a unique number to every possible state, and RL learns to choose an action number from 0-5 where:

 - 0 = south
 - 1 = north
 - 2 = east
 - 3 = west
 - 4 = pickup
 - 5 = dropoff

Recall that the 500 states correspond to a encoding of the taxi's location, the passenger's location, and the destination location.

Reinforcement Learning will learn a mapping of states to the optimal action to perform in that state by exploration, i.e. the agent explores the environment and takes actions based off rewards defined in the environment.

The optimal action for each state is the action that has the highest cumulative long-term reward.



![](./Reinforcement_Learning_Taxi_Env.width-1200.png)

We can actually pass in the details shown in the pic above

In [67]:
env = gym.make("Taxi-v3").env
env.reset()

state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.env.s = state
env.render()

State: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



We are using our illustration's coordinates to generate a number corresponding to a state between 0 and 499, which turns out to be 328 for our illustration's state.

Then we can set the environment's state manually with env.env.s using that encoded number. You can play around with the numbers and you'll see the taxi, passenger, and destination move around.

In [75]:
# env.env.s = 123
# env.render()


### The Reward Table (Q Table)
When the Taxi environment is created, there is an initial Reward table that's also created, called `P`. We can think of it like a matrix that has the number of states as rows and number of actions as columns, i.e. a  matrix.

Since every state is in this matrix, we can see the default reward values assigned to our illustration's state:

In [72]:
env.P[328]

# {0: [(1.0, 428, -1, False)],
#  1: [(1.0, 228, -1, False)],
#  2: [(1.0, 348, -1, False)],
#  3: [(1.0, 328, -1, False)],
#  4: [(1.0, 328, -10, False)],
#  5: [(1.0, 328, -10, False)]}

# This dictionary has the structure: {action: [(probability, nextstate, reward, done)]}.
# 0, 1, 2, 3, 4, 5 = DOWN, UP, RIGHT, LEFT, PICKUP, DROP-OFF 


{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

This dictionary has the structure: `{action: [(probability, nextstate, reward, done)]}`.

A few things to note:

 - The 0-5 corresponds to the actions (south, north, east, west, pickup, dropoff) the taxi can perform at our current state in the illustration.
 - In this `env`, probability is always 1.0.
 - The nextstate is the state we would be in if we take the action at this index of the dict
 - All the movement actions have a -1 reward and the pickup/dropoff actions have -10 reward in this particular state. If we are in a state where the taxi has a passenger and is on top of the right destination, we would see a reward of 20 at the dropoff action (5)
 - **done** is used to tell us when we have successfully dropped off a passenger in the right location. Each successfull dropoff is the end of an episode

Note: if our agent chose to explore action two (2) in this state it would be going East into a wall. The source code has made it impossible to actually move the taxi across a wall, so if the taxi chooses that action, it will just keep accruing -1 penalties, which affects the **long-term reward**.

### Solving the environment without Reinforcement Learning

Let's see what would happen if we try to brute-force our way to solving the problem without RL.

Since we have our P table for default rewards in each state, we can try to have our taxi navigate just using that.

We'll create an infinite loop which runs until one passenger reaches one destination (one episode), or in other words, when the received reward is 20. 

The `env.action_space.sample()` method automatically selects one random action from set of all possible actions.

Let's see what happens:

In [90]:
env.s = 328  # set environment to illustration's state

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode="ansi"),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs)) # 200
print("Penalties incurred: {}".format(penalties)) # 76


Timesteps taken: 1
Penalties incurred: 0


In [92]:
# Implementing Q-learning in python
# initialise Q-table with 500 x 6 matrix of zeroes
state_space = env.observation_space.n
print("There are ", state_space, " possible states")
action_space = env.action_space.n
print("There are ", action_space, " possible actions")


q_table = np.zeros((state_space, action_space)) 
print(q_table.shape) # (500, 6)
print(q_table)

There are  500  possible states
There are  6  possible actions
(500, 6)
[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


In [97]:
# DEFINE HYPERPARAMTERS

total_episodes = 50000        # Total episodes
total_test_episodes = 100     # Total test episodes
max_steps = 99                # Max steps per episode

learning_rate = 0.7           # Learning rate
gamma = 0.618                 # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability 
decay_rate = 0.01             # Exponential decay rate for exploration prob



### Define the epsilon-greedy policy

Epsilon-Greedy is a policy that handles the exploration/[exploitation trade-off](https://medium.com/@thomassimonini/an-introduction-to-deep-reinforcement-learning-17a565999c0c?source=friends_link&sk=1b1121ae5d9814a09ca38b47abc7dc61).

### The idea

Epsilon Greedy:

- We do **exploitation** (aka our agent selects the action with the highest state-action pair value) with a prob of *1 - ɛ* : 
- We do **exploration** (trying random action) *with probability ɛ*:

As the training goes on, **we progressively reduce the epsilon value** since we will **need less and less exploration and more exploitation**.

In [98]:
### Define the epsilon-greedy policy

# Epsilon-Greedy is a policy that handles the exploration/[exploitation trade-off](https://medium.com/@thomassimonini/an-introduction-to-deep-reinforcement-learning-17a565999c0c?source=friends_link&sk=1b1121ae5d9814a09ca38b47abc7dc61).
# Epsilon Greedy:
# - We do **exploitation** (aka our agent selects the action with the highest state-action pair value) with a prob of *1 - ɛ* : 
# - We do **exploration** (trying random action) *with probability ɛ*:
# As the training goes on, **we progressively reduce the epsilon value** since we will **need less and less exploration and more exploitation**.

def epsilon_greedy_policy(q_table, state):
    
  # if random number > greater than epsilon --> exploitation 
  # remember epsilon starts at 1 and decals to 0.001 at a rate of 0.01
  if(random.uniform(0,1) > epsilon):

      # the following line selects the index of the highest point value of the available moves for the state at the index of the state
    action = np.argmax(q_table[state])

  # else --> exploration (i.e. random choice from action_space)
  else:
    action = env.action_space.sample()
  
  return action

In [101]:
# implement the Q learning algorithm:

for episode in range(total_episodes):
    # Reset the environment
    state = env.reset()
    step = 0
    done = False

    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)
    
    for step in range(max_steps):
        
        action = epsilon_greedy_policy(q_table, state) # returns action (num 0 - 5)

        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done, info = env.step(action) 

        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        q_table[state][action] = q_table[state][action] + learning_rate * (reward + gamma * 
                                    np.max(q_table[new_state]) - q_table[state][action])      
        # If done : finish episode
        if done == True: 
            break
        
        # Our new state is state
        state = new_state



In [104]:
import time
rewards = []

frames = []
for episode in range(total_test_episodes):
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0
    # print("****************************************************")
    # print("EPISODE ", episode)
    for step in range(max_steps):
        # env.render()     
        # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(q_table[state][:])
        new_state, reward, done, info = env.step(action)
        total_rewards += reward
        
        if done:
            rewards.append(total_rewards)
            #print ("Score", total_rewards)
            break
        state = new_state
env.close()
print ("Score over time: " +  str(sum(rewards)/total_test_episodes))

Score over time: 7.61


# THE ABOVE WAS USING https://colab.research.google.com/gist/simoninithomas/466c81aa1c2a07dd14793240c6d033c5/q-learning-with-taxi-v3.ipynb#scrollTo=WlJYOh0yBHZO
# THE BELOW IS ANOTHER VERSION

In [ ]:
env = gym.make("Taxi-v3").env
env.reset()

state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.env.s = state
env.render()

We can now create the training algorithm that will update this Q-table as the agent explores the environment over thousands of episodes.

In the first part of `while not done`, we decide whether to pick a random action or to exploit the already computed Q-values. This is done simply by using the `epsilon` value and comparing it to the `random.uniform(0, 1)` function, which returns an arbitrary number between 0 and 1.

We execute the chosen action in the environment to obtain the `next_state` and the `reward` from performing the action. After that, we calculate the maximum Q-value for the actions corresponding to the `next_state`, and with that, we can easily update our Q-value to the `new_q_value`:

In [106]:
q_table = np.zeros([env.observation_space.n, env.action_space.n])

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()
    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.



In [107]:
# Now that the Q-table has been established over 100,000 episodes, let's see what the Q-values are at our illustration's state:
q_table[328]

# array([ -2.40678459,  -2.27325184,  -2.41361289,  -2.36065942, -10.99713999, -10.85964757])
# The max Q-value is "north" (-1.971), so it looks like Q-learning has effectively learned the best action to take in our illustration's state!

array([ -2.40678459,  -2.27325184,  -2.41361289,  -2.36065942,
       -10.99713999, -10.85964757])

In [109]:
#  Evaluating the agent
# We don't need to explore actions any further, so now the next action is always selected using the best Q-value:

total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 12.75
Average penalties per episode: 0.0


We can see from the evaluation, the agent's performance improved significantly and it incurred no penalties, which means it performed the correct pickup/dropoff actions with 100 different passengers.

Hyperparameters and optimizations
The values of `alpha`, `gamma`, and `epsilon` were mostly based on intuition and some "hit and trial", but there are better ways to come up with good values.

Ideally, all three should decrease over time because as the agent continues to learn, it actually builds up more resilient priors;

ALPHA : (the learning rate) should decrease as you continue to gain a larger and larger knowledge base.
GAMMA : as you get closer and closer to the deadline, your preference for near-term reward should increase, as you won't be around long enough to get the long-term reward, which means your gamma should decrease.
EPSILON : as we develop our strategy, we have less need of exploration and more exploitation to get more utility from our policy, so as trials increase, epsilon should decrease.